In [ ]:
gdpr_article_5_text = """
                      1. Personal data shall be:
                      a) processed lawfully, fairly and in a transparent manner in relation to the data subject (‘lawfulness, fairness and transparency’);
                      b) collected for specified, explicit and legitimate purposes and not further processed in a manner that is incompatible with those purposes; further processing for archiving purposes in the public interest, scientific or historical research purposes or statistical purposes shall, in accordance with Article 89(1), not be considered to be incompatible with the initial purposes (‘purpose limitation’);
                      c) adequate, relevant and limited to what is necessary in relation to the purposes for which they are processed (‘data minimisation’);
                      d) accurate and, where necessary, kept up to date; every reasonable step must be taken to ensure that personal data that are inaccurate, having regard to the purposes for which they are processed, are erased or rectified without delay (‘accuracy’);
                      e) kept in a form which permits identification of data subjects for no longer than is necessary for the purposes for which the personal data are processed; personal data may be stored for longer periods insofar as the personal data will be processed solely for archiving purposes in the public interest, scientific or historical research purposes or statistical purposes in accordance with Article 89(1) subject to implementation of the appropriate technical and organisational measures required by this Regulation in order to safeguard the rights and freedoms of the data subject (‘storage limitation’);
                      f) processed in a manner that ensures appropriate security of the personal data, including protection against unauthorised or unlawful processing and against accidental loss, destruction or damage, using appropriate technical or organisational measures (‘integrity and confidentiality’).
                      2. The controller shall be responsible for, and be able to demonstrate compliance with, paragraph 1 (‘accountability’).
                      """

In [ ]:
!unzip /content/acl-coling-2014-corpus.zip -d /content/acl_corpus

Archive:  /content/acl-coling-2014-corpus.zip
   creating: /content/acl_corpus/corpus/
  inflating: /content/acl_corpus/corpus/9gag.xml  
   creating: /content/acl_corpus/__MACOSX/
   creating: /content/acl_corpus/__MACOSX/corpus/
  inflating: /content/acl_corpus/__MACOSX/corpus/._9gag.xml  
  inflating: /content/acl_corpus/corpus/about_abc_net_au.xml  
  inflating: /content/acl_corpus/__MACOSX/corpus/._about_abc_net_au.xml  
  inflating: /content/acl_corpus/corpus/about_ask.xml  
  inflating: /content/acl_corpus/__MACOSX/corpus/._about_ask.xml  
  inflating: /content/acl_corpus/corpus/about_deviantart.xml  
  inflating: /content/acl_corpus/__MACOSX/corpus/._about_deviantart.xml  
  inflating: /content/acl_corpus/corpus/about_officemax.xml  
  inflating: /content/acl_corpus/__MACOSX/corpus/._about_officemax.xml  
  inflating: /content/acl_corpus/corpus/about_pinterest.xml  
  inflating: /content/acl_corpus/__MACOSX/corpus/._about_pinterest.xml  
  inflating: /content/acl_corpus/corpus/

In [ ]:
import xml.etree.ElementTree as ET
import spacy
import os

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    # Lowercase and replace newline characters
    text = text.lower().replace('\n', ' ')
    text = ' '.join(text.split())

    # Use spaCy for further processing
    doc = nlp(text)
    clean_text = ' '.join([token.lemma_ for token in doc if not token.is_punct])

    return clean_text


preprocessed_policies = []
directory = './acl_corpus/corpus/'

for filename in os.listdir(directory):
    if filename.endswith('.xml'):
        path = os.path.join(directory, filename)
        tree = ET.parse(path)
        root = tree.getroot()

        # Extracting text from each SECTION/SUBTEXT
        for section in root.findall('.//SECTION/SUBTEXT'):
            if section.text:
                processed_text = preprocess_text(section.text)
                if processed_text.strip():  # this ensures the string is not just whitespace
                    preprocessed_policies.append(processed_text)
                else:
                    print(f"Empty or non-meaningful section found in {filename}, section skipped.")

print(f"Number of processed policy sections: {len(preprocessed_policies)}")

Empty or non-meaningful section found in corporate_yp.xml, section skipped.
Number of processed policy sections: 10500


In [ ]:
!pip install -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 20.9 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [ ]:
!unzip /content/saved_model.zip -d /content/saved_model

Archive:  /content/saved_model.zip
   creating: /content/saved_model/saved_model/
  inflating: /content/saved_model/saved_model/special_tokens_map.json  
  inflating: /content/saved_model/saved_model/merges.txt  
  inflating: /content/saved_model/saved_model/vocab.json  
  inflating: /content/saved_model/saved_model/config.json  
  inflating: /content/saved_model/saved_model/tokenizer_config.json  
  inflating: /content/saved_model/saved_model/model.safetensors  


In [ ]:
from transformers import GPT2ForSequenceClassification, GPT2Tokenizer
import torch
import numpy as np

model = GPT2ForSequenceClassification.from_pretrained('./saved_model/saved_model')
tokenizer = GPT2Tokenizer.from_pretrained('./saved_model/saved_model')

model.config.output_hidden_states = True

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    model.resize_token_embeddings(len(tokenizer))

def get_gpt_embeddings(texts, model, tokenizer):
    embeddings = []
    for text in texts:
        inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512, add_special_tokens=True)
        # Obtain hidden states
        with torch.no_grad():
            outputs = model(**inputs)
            hidden_states = outputs.hidden_states[-1]
        # Take the mean of all token embeddings for a text representation
        text_embedding = hidden_states.mean(dim=1)
        embeddings.append(text_embedding)
    embeddings = torch.stack(embeddings).squeeze()
    return embeddings.numpy()

# Example use
gdpr_text = [gdpr_article_5_text]
policy_embeddings = get_gpt_embeddings(preprocessed_policies, model, tokenizer)
gdpr_embeddings = get_gpt_embeddings(gdpr_text, model, tokenizer)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate semantic similarity
similarities = cosine_similarity(policy_embeddings, gdpr_embeddings.reshape(1, -1))
similar_policies_indices = np.where(similarities > 0.9)[0]

print(f"Number of policies potentially aligning with GDPR Article 5: {len(similar_policies_indices)}")


Number of policies potentially aligning with GDPR Article 5: 10005


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate semantic similarity
similarities = cosine_similarity(policy_embeddings, gdpr_embeddings.reshape(1, -1))
similar_policies_indices = np.where(similarities > 0.98)[0]

print(f"Number of policies potentially aligning with GDPR Article 5: {len(similar_policies_indices)}")


Number of policies potentially aligning with GDPR Article 5: 2281


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Calculate semantic similarity
similarities = cosine_similarity(policy_embeddings, gdpr_embeddings.reshape(1, -1))
similar_policies_indices = np.where(similarities > 0.99)[0]

print(f"Number of policies potentially aligning with GDPR Article 5: {len(similar_policies_indices)}")


Number of policies potentially aligning with GDPR Article 5: 1


- **Embedding Models:** Other than *SBERT*, I can try other models like *GPT-3* for context-aware embeddings, *FastText* for handling out-of-vocabulary words, or domain-specific models that might be trained on legal or policy-related corpora.

- **Similarity Measures:** Beyond *cosine similarity*, explore other measures like *Euclidean distance* or *Manhattan distance* for comparing embeddings.

- **Clustering Techniques:** Apply unsupervised clustering (e.g., *K-means, DBSCAN*) to group policies based on their embeddings and analyze clusters for GDPR compliance themes.

- **Dimensionality Reduction:** Use techniques like *PCA* or *t-SNE* to reduce the dimensionality of your embeddings before applying similarity measures or clustering, which might reveal different patterns.

- **Threshold Tuning:** Experiment with different *thresholds* for considering a policy potentially compliant based on similarity scores.

- **Text Preprocessing Variations:** Test the impact of different *preprocessing steps*, such as including/excluding stopwords, using stemming versus lemmatization, or experimenting with n-grams.

- **Segmentation Strategies:** Instead of analyzing entire documents, try segmenting policies into smaller *sections or paragraphs* and assess their individual compliance.